# This book is identical to RNN_corn_weekly_weather.ipynb, except for the generation of test set predictions after validation set predictions

### CONTENTS
### 1 Imports
### 2 Arrange datasets
### 3 Define models
### 4 Train models

# 1 Imports

In [1]:
# RNN imports

import tensorflow as tf

from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN

from keras.models import load_model

from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#General imports

import numpy as np
import pandas as pd

import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib.dates

from datetime import datetime

from google.colab import drive
import json

In [3]:
# Import weekly data

data_url = 'https://raw.githubusercontent.com/ptraver/data/main/corn_weekly.csv'
corn_df = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

# fix bad name
corn_df.columns = ['value']

# take only 1974 - 2017
corn_df = corn_df['1974-01-01':'2017-12-31']

# corn_df needs to lose last value becuase during process of VMD, the last value is lost
corn_df = corn_df[:-1]

# view loaded data
corn_df.tail(2)

,value
date,
2017-12-10,339.30
2017-12-17,338.85


In [4]:
# Import weather sets
data_url = 'https://raw.githubusercontent.com/ptraver/data/main/iowa_weather_train.csv'
iowa_weather_train = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='DATE')

data_url = 'https://raw.githubusercontent.com/ptraver/data/main/iowa_weather_valid.csv'
iowa_weather_valid = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='DATE')

data_url = 'https://raw.githubusercontent.com/ptraver/data/main/iowa_weather_test.csv'
iowa_weather_test = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='DATE')

# 2 Arrange datasets

In [5]:
# Split data so that valid and test do not feature in decisions
from sklearn.model_selection import train_test_split

true_train_split, out_of_sample = train_test_split(corn_df, test_size=0.2, shuffle=False)
true_valid_split, true_test_split = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

In [6]:
# Scale data between 0 and 1
from sklearn.preprocessing import MinMaxScaler

true_scaler = MinMaxScaler()
true_train_scaled = true_scaler.fit_transform(true_train_split)
true_valid_scaled = true_scaler.transform(true_valid_split)
true_test_scaled = true_scaler.transform(true_test_split)

In [7]:
true_train = pd.DataFrame(true_train_scaled, columns=true_train_split.columns, index=true_train_split.index)
true_valid = pd.DataFrame(true_valid_scaled, columns=true_valid_split.columns, index=true_valid_split.index)
true_test = pd.DataFrame(true_test_scaled, columns=true_test_split.columns, index=true_test_split.index)

In [8]:
# Double-check that indices are equal
print(true_train.index.equals(iowa_weather_train.index))
print(true_valid.index.equals(iowa_weather_valid.index))
print(true_test.index.equals(iowa_weather_test.index))

True
True
True


In [9]:
# Join weather and price data
train_weather = true_train.join(iowa_weather_train)
valid_weather = true_valid.join(iowa_weather_valid)
test_weather = true_test.join(iowa_weather_test)

In [10]:
# Set input shape variable
n_inputs = 4

In [11]:
# Get sets in order
train_shaped = TimeseriesGenerator(train_weather, true_train.values, length=n_inputs, batch_size=1)
valid_shaped = TimeseriesGenerator(valid_weather, true_valid.values, length=n_inputs, batch_size=1)
test_shaped = TimeseriesGenerator(test_weather, true_test.values, length=n_inputs, batch_size=1)

# 3 Define models

In [ ]:
# Set shape variables
L1_units = 8
L2_units = 12
L3_units = 8

# Set number of features
n_features = 4

In [ ]:
# Set other variables
activation = 'relu'
dropout = 0
optimizer = 'adam'
loss = 'mse'

In [ ]:
# Define vmd model definition function

def define_model():
  RNNregressor = Sequential()
  RNNregressor.add(SimpleRNN(units=L1_units, input_shape=(n_inputs, n_features), activation=activation, dropout=dropout, return_sequences=True))
  RNNregressor.add(SimpleRNN(units=L2_units, input_shape=(n_inputs, n_features), activation=activation, dropout=dropout, return_sequences=True))
  RNNregressor.add(SimpleRNN(units=L3_units, input_shape=(n_inputs, n_features), activation=activation, dropout=dropout))
  RNNregressor.add(Dense(units=1))
  RNNregressor.compile(optimizer=optimizer, loss=loss)
  return RNNregressor

# 4 Train models

In [ ]:
# Set variables
EPOCHS = 50

In [12]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Set checkpoint callback
filepath = 'true_RNN_weather_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
RNNregressor = define_model()

# Train model
with tf.device('/device:GPU:0'):
  RNNregressor.fit(train_shaped,
                   epochs=EPOCHS,
                   validation_data=valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/50
1831/1831 [==============================] - ETA: 0s - loss: 0.0216INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_01.model/assets


INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_01.model/assets


1831/1831 [==============================] - 31s 16ms/step - loss: 0.0216 - val_loss: 0.1633
Epoch 2/50
1830/1831 [============================>.] - ETA: 0s - loss: 0.0045INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_02.model/assets


INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_02.model/assets


1831/1831 [==============================] - 30s 16ms/step - loss: 0.0045 - val_loss: 0.0150
Epoch 3/50
1830/1831 [============================>.] - ETA: 0s - loss: 0.0018INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_03.model/assets


...

1831/1831 [==============================] - 29s 16ms/step - loss: 3.2269e-04 - val_loss: 0.0042
Epoch 49/50
1829/1831 [============================>.] - ETA: 0s - loss: 3.2475e-04INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_49.model/assets


INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_49.model/assets


1831/1831 [==============================] - 29s 16ms/step - loss: 3.2689e-04 - val_loss: 0.0019
Epoch 50/50
1829/1831 [============================>.] - ETA: 0s - loss: 3.3948e-04INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_50.model/assets


INFO:tensorflow:Assets written to: models/true_RNN_weather_valid_50.model/assets


1831/1831 [==============================] - 29s 16ms/step - loss: 3.3939e-04 - val_loss: 0.0025


In [13]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'true_RNN_weather_valid_24'

In [ ]:
# Save best model to Drive
true_RNN_weather_model = load_model(f'models/{best_model}.model')
true_RNN_weather_model.save(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
true_fc = true_RNN_weather_model.predict(valid_shaped)

In [ ]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/true_RNN_weather_valid.npy', true_rescaled.flatten())

In [ ]:
#### TEST SET ####

In [14]:
# Load the model
true_RNN_weather_model = load_model(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [15]:
# Use best model to generate emd test set forecasts
true_fc = true_RNN_weather_model.predict(test_shaped)

In [16]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [19]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/true_RNN_weather_test.npy', true_rescaled.flatten())

# End